<a href="https://colab.research.google.com/github/AdityaKareer/AdityaKareer/blob/main/MLP_Markov_model_5_epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
from PIL import Image
from PIL import Image
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

class markovImg:
    def __init__(self, data):
        self.npimg = data
        self.imgH = self.npimg.shape[0] #height
        self.imgL = self.npimg.shape[1] #Length
        self.nbr = np.full((self.imgH, self.imgL), -1)
        for i in range(self.imgH):
            for j in range(self.imgL):
                self.nbr[i][j] = self.neighbor(i, j)

    @classmethod
    def link(self, filepath):
        return markovImg(np.array(Image.open(filepath)))

    def pxl(self, x, y):
        return self.npimg[x][y]

    def pxlc(self, x, y):
        return color(self.npimg[x][y])

    def h(self):
        return self.imgH

    def l(self):
        return self.imgL

    def neighbor(self, x, y):
        n = 0
        try:
            nlist = [
                self.pxlc(x + 1, y + 1),
                self.pxlc(x, y + 1),
                self.pxlc(x - 1, y + 1),
                self.pxlc(x + 1, y),
                self.pxlc(x - 1, y),
                self.pxlc(x + 1, y - 1),
                self.pxlc(x, y - 1),
                self.pxlc(x - 1, y - 1)
            ]
            n2 = [i for i in nlist if i > 4]
            if (n2 == [] and self.pxlc(x, y) < 5):
                n = (
                    nlist.count(0)
                    + 10 * nlist.count(1)
                    + 100 * nlist.count(2)
                    + 1000 * nlist.count(3)
                    + 10000 * nlist.count(4)
                    + 100000 * color(self.pxl(x, y))
                )
                return n
            else:
                return -1
        except:
            return -1

def color(x):
    if ((x == blue).all()):
        return 0
    elif ((x == red).all()):
        return 1
    elif ((x == lgreen).all()):
        return 2
    elif ((x == dgreen).all()):
        return 3
    elif ((x == yellow).all()):
        return 4
    else:
        return 5

def colorRet(x):
    if x == 0:
        return blue  # Water
    elif x == 1:
        return red  # Urban
    elif x == 2:
        return lgreen  # Water Vegetation
    elif x == 3:
        return dgreen  # Vegetation
    elif x == 4:
        return yellow  # Barren lands
    elif x == 5:
        return white  # Other

def preprocess_img(img):
    img_data = img.npimg
    img_data = img_data.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
    img_data = img_data.reshape(-1, 3)  # Flatten the image data
    img_labels = np.array([colorRet(color) for row in img.nbr for color in row])

    label_encoder = LabelEncoder()
    img_labels_encoded = label_encoder.fit_transform(img_labels)
    img_labels_onehot = to_categorical(img_labels_encoded)

    return img_data, img_labels_onehot, label_encoder


def CAgen(img1, img2, img3):
    if img1.h() == img2.h() == img3.h() and img1.l() == img2.l() == img3.l():
        occ = {}
        for i in range(img1.h()):
            for j in range(img1.l()):
                if img1.nbr[i][j] != -1 and img2.nbr[i][j] != -1:
                    k = img1.nbr[i][j]
                    k2 = img2.nbr[i][j]
                    if k in occ:
                        if img2.pxlc(i, j) < 5:
                            occ[k][img2.pxlc(i, j)] = occ[k][img2.pxlc(i, j)] + 1
                    else:
                        occ[k] = [0, 0, 0, 0, 0]
                        occ[k][img2.pxlc(i, j)] = occ[k][img2.pxlc(i, j)] + 1
                    if k2 in occ:
                        if img3.pxlc(i, j) < 5:
                            occ[k2][img3.pxlc(i, j)] = occ[k2][img3.pxlc(i, j)] + 1
                    else:
                        occ[k2] = [0, 0, 0, 0, 0]
                        if img3.pxlc(i, j) < 5:
                            occ[k2][img3.pxlc(i, j)] = occ[k2][img3.pxlc(i, j)] + 1

        for i in occ:
            s = sum(occ[i])
            if s > 1:
                for j in range(len(occ[i])):
                    occ[i][j] = occ[i][j] / s
                    if j > 0:
                        occ[i][j] += occ[i][j - 1]

        return occ

# Example usage:
img2010 = markovImg.link("/content/drive/MyDrive/Maps/2010.bmp")
img2015 = markovImg.link("/content/drive/MyDrive/Maps/2015.bmp")
img2020 = markovImg.link("/content/drive/MyDrive/Maps/2020.bmp")

occ = CAgen(img2010, img2015, img2020)

# Assuming you have paths to the image files for the years 2000, 2010, and 2020
img2000 = markovImg.link("/content/drive/MyDrive/10yeargap/2000.bmp")
img2010 = markovImg.link("/content/drive/MyDrive/10yeargap/2010.bmp")
img2020 = markovImg.link("/content/drive/MyDrive/10yeargap/2020.bmp")

# Now, you can proceed with the preprocessing
img2000_data, img2000_labels, label_encoder = preprocess_img(img2000)
img2010_data, img2010_labels, _ = preprocess_img(img2010)
img2020_data, _, _ = preprocess_img(img2020)

def create_mlp_model(input_shape, num_classes):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Set up the input shape and number of classes for the MLP model
input_shape = (img2000_data.shape[1],)
num_classes = len(label_encoder.classes_)

# Create and train the MLP model
mlp_model = create_mlp_model(input_shape, num_classes)
mlp_model.fit(img2000_data, img2000_labels, epochs=5, batch_size=32, validation_split=0.2)

# Make predictions for the year 2020 using the trained MLP model
predictions = mlp_model.predict(img2020_data)
predicted_labels_encoded = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded)

# Reshape predicted labels to match the image dimensions
predicted_img = predicted_labels.reshape((img2020.h(), img2020.l()))

# Generate and display the predicted image
predicted_img_obj = markovImg(predicted_img)
CAgen(img2020, predicted_img_obj, img2020)


In [ ]:
from PIL import Image
import numpy as np
import random
!pip install -U -q PyDrive
!pip install tabulate
from tabulate import tabulate
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
